# GraphCast updated

In [1]:
import dataclasses
import datetime
import functools
import math
import re
from typing import Optional

import cartopy.crs as ccrs
from google.cloud import storage
from graphcast import autoregressive
from graphcast import casting
from graphcast import checkpoint
from graphcast import data_utils
from graphcast import graphcast
from graphcast import normalization
from graphcast import rollout
from graphcast import xarray_jax
from graphcast import xarray_tree
from graphcast import model_utils
from IPython.display import HTML
import ipywidgets as widgets
import haiku as hk
import jax
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
import xarray
import warnings

warnings.filterwarnings('ignore')

In [2]:
def parse_file_parts(file_name):
  return dict(part.split("-", 1) for part in file_name.split("_"))


gcs_client = storage.Client.create_anonymous_client()
gcs_bucket = gcs_client.get_bucket("dm_graphcast")
dir_prefix = "graphcast/"

with gcs_bucket.blob(f"{dir_prefix}params/GraphCast_small - ERA5 1979-2015 - resolution 1.0 - pressure levels 13 - mesh 2to5 - precipitation input and output.npz").open("rb") as f:
    ckpt = checkpoint.load(f, graphcast.CheckPoint)

params = ckpt.params
state = {}
model_config = ckpt.model_config
task_config = ckpt.task_config

name = 'source-era5_date-2022-01-01_res-1.0_levels-13_steps-04.nc'
with gcs_bucket.blob(f"{dir_prefix}dataset/{name}").open("rb") as f:
    example_batch = xarray.load_dataset(f).compute()
example_batch

<xarray.Dataset> Size: 132MB
Dimensions:                       (lon: 360, lat: 181, level: 13, time: 6,
                                   batch: 1)
Coordinates:
  * lon                           (lon) float32 1kB 0.0 1.0 2.0 ... 358.0 359.0
  * lat                           (lat) float32 724B -90.0 -89.0 ... 89.0 90.0
  * level                         (level) int32 52B 50 100 150 ... 850 925 1000
  * time                          (time) timedelta64[ns] 48B 00:00:00 ... 1 d...
    datetime                      (batch, time) datetime64[ns] 48B 2022-01-01...
Dimensions without coordinates: batch
Data variables: (12/14)
    geopotential_at_surface       (lat, lon) float32 261kB 2.735e+04 ... -0.0...
    land_sea_mask                 (lat, lon) float32 261kB 1.0 1.0 ... 0.0 0.0
    2m_temperature                (batch, time, lat, lon) float32 2MB 250.7 ....
    mean_sea_level_pressure       (batch, time, lat, lon) float32 2MB 9.931e+...
    10m_v_component_of_wind       (batch, time, lat, lon) float32 2MB -0.4393...
    10m_u_component_of_wind       (batch, time, lat, lon) float32 2MB 1.309 ....
    ...                            ...
    temperature                   (batch, time, level, lat, lon) float32 20MB ...
    geopotential                  (batch, time, level, lat, lon) float32 20MB ...
    u_component_of_wind           (batch, time, level, lat, lon) float32 20MB ...
    v_component_of_wind           (batch, time, level, lat, lon) float32 20MB ...
    vertical_velocity             (batch, time, level, lat, lon) float32 20MB ...
    specific_humidity             (batch, time, level, lat, lon) float32 20MB ...

In [3]:
eval_inputs, eval_targets, eval_forcings = data_utils.extract_inputs_targets_forcings(
    example_batch, input_duration='24h', target_lead_times='6h',
    **{k: v for k, v in task_config.items() if k != "input_duration"})

with gcs_bucket.blob(dir_prefix+"stats/diffs_stddev_by_level.nc").open("rb") as f:
    diffs_stddev_by_level = xarray.load_dataset(f).compute()
with gcs_bucket.blob(dir_prefix+"stats/mean_by_level.nc").open("rb") as f:
    mean_by_level = xarray.load_dataset(f).compute()
with gcs_bucket.blob(dir_prefix+"stats/stddev_by_level.nc").open("rb") as f:
    stddev_by_level = xarray.load_dataset(f).compute()

In [4]:
def construct_wrapped_graphcast(
    model_config: graphcast.ModelConfig,
    task_config: graphcast.TaskConfig):
  """Constructs and wraps the GraphCast Predictor."""
  # Deeper one-step predictor.
  predictor = graphcast.GraphCast(model_config, task_config)

  # Modify inputs/outputs to `graphcast.GraphCast` to handle conversion to
  # from/to float32 to/from BFloat16.
  predictor = casting.Bfloat16Cast(predictor)

  # Modify inputs/outputs to `casting.Bfloat16Cast` so the casting to/from
  # BFloat16 happens after applying normalization to the inputs/targets.
  predictor = normalization.InputsAndResiduals(
      predictor,
      diffs_stddev_by_level=diffs_stddev_by_level,
      mean_by_level=mean_by_level,
      stddev_by_level=stddev_by_level)

  # Wraps everything so the one-step model can produce trajectories.
  predictor = autoregressive.Predictor(predictor, gradient_checkpointing=True)
  return predictor


@hk.transform_with_state
def run_forward(model_config, task_config, inputs, targets_template, forcings):
  global predictor
  predictor = construct_wrapped_graphcast(model_config, task_config)
  return predictor(inputs, targets_template=targets_template, forcings=forcings)


@hk.transform_with_state
def loss_fn(model_config, task_config, inputs, targets, forcings):
  predictor = construct_wrapped_graphcast(model_config, task_config)
  loss, diagnostics = predictor.loss(inputs, targets, forcings)
  return xarray_tree.map_structure(
      lambda x: xarray_jax.unwrap_data(x.mean(), require_jax=True),
      (loss, diagnostics))

def grads_fn(params, state, model_config, task_config, inputs, targets, forcings):
  def _aux(params, state, i, t, f):
    (loss, diagnostics), next_state = loss_fn.apply(
        params, state, jax.random.PRNGKey(0), model_config, task_config,
        i, t, f)
    return loss, (diagnostics, next_state)
  (loss, (diagnostics, next_state)), grads = jax.value_and_grad(
      _aux, has_aux=True)(params, state, inputs, targets, forcings)
  return loss, diagnostics, next_state, grads

# Jax doesn't seem to like passing configs as args through the jit. Passing it
# in via partial (instead of capture by closure) forces jax to invalidate the
# jit cache if you change configs.
def with_configs(fn):
  return functools.partial(
      fn, model_config=model_config, task_config=task_config)

# Always pass params and state, so the usage below are simpler
def with_params(fn):
  return functools.partial(fn, params=params, state=state)

# Our models aren't stateful, so the state is always empty, so just return the
# predictions. This is requiredy by our rollout code, and generally simpler.
def drop_state(fn):
  return lambda **kw: fn(**kw)[0]

init_jitted = jax.jit(with_configs(run_forward.init))

loss_fn_jitted = drop_state(with_params(jax.jit(with_configs(loss_fn.apply))))
grads_fn_jitted = with_params(jax.jit(with_configs(grads_fn)))
run_forward_jitted = drop_state(with_params(jax.jit(with_configs(
    run_forward.apply))))

In [5]:
predictions, embeddings, latent = rollout.chunked_prediction(
    run_forward_jitted,
    rng=jax.random.PRNGKey(0),
    inputs=eval_inputs,
    targets_template=eval_targets * np.nan,
    forcings=eval_forcings)

In [ ]:
import torch
import torch.nn as nn
import numpy as np

from tqdm import tqdm


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [17]:
net = nn.Sequential(nn.Linear(1024, 2048),
                    nn.ReLU(),
                    nn.Dropout(0.3),
                    nn.Linear(2048, 2048),
                    nn.ReLU(),
                    nn.Dropout(0.3),
                    nn.Linear(2048, 1024),
                    nn.ReLU(),
                    nn.Dropout(0.3),
                    nn.Linear(1024, 512),
                    nn.ReLU(),
                    nn.Dropout(0.3),
                    nn.Linear(512, 83)).to(device)
criterion = nn.MSELoss()
optim = torch.optim.Adam([p for p in net.parameters() if p.requires_grad], lr=1e-4, weight_decay=1e-3)

In [28]:
x[:,:512].shape

torch.Size([65160, 512])

In [3]:
for k, v in torch.load('data.pt').items():
    globals()[k] = v.to(device)
targets = targets.reshape(-1, 83)

In [18]:
bs = 512
net.train()
for i in tqdm(range(100)):
    indices = np.random.permutation(np.arange(targets.shape[0]))
    for j in range(0, len(targets), bs):
        idx = indices[i:i+bs]
        y = net(x[idx]) + old[idx]
        loss = criterion(y, targets[idx])
        loss.backward()
        optim.step()
        loss = loss.detach().cpu()
    if i % 5 == 0:
        print(loss)

  1%|▊                                                                                 | 1/100 [00:03<05:56,  3.60s/it]

tensor(7390.6572, device='cuda:0', grad_fn=<MseLossBackward0>)


  6%|████▉                                                                             | 6/100 [00:20<05:27,  3.49s/it]

tensor(4802.6069, device='cuda:0', grad_fn=<MseLossBackward0>)


 11%|████████▉                                                                        | 11/100 [00:38<05:08,  3.46s/it]

...

 91%|█████████████████████████████████████████████████████████████████████████▋       | 91/100 [05:19<00:32,  3.58s/it]

tensor(4307.8457, device='cuda:0', grad_fn=<MseLossBackward0>)


 96%|█████████████████████████████████████████████████████████████████████████████▊   | 96/100 [05:37<00:14,  3.54s/it]

tensor(3604.7363, device='cuda:0', grad_fn=<MseLossBackward0>)


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [05:51<00:00,  3.52s/it]
